# Advent of Code 2025

> Computer science is no more about computers than astronomy is about telescopes.
>
-- Edsger W. Dijkstra

This year there will only be [12 problems](https://adventofcode.com/2025/about#faq_num_days) rather than 24. It won't be any less enjoyable!

## Imports and definitions

In [1]:
from urllib import request
from itertools import accumulate, combinations, zip_longest, pairwise
from functools import cache, reduce
from math import lcm, prod, sumprod
from bisect import bisect_right, bisect_left
from operator import mul, add, xor
from collections import Counter
from dataclasses import dataclass
from scipy.spatial import Delaunay
from scipy.optimize import milp


def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2025/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: Secret Entrance](https://adventofcode.com/2025/day/1)

In [2]:
def hits(move, start):
    revolutions = abs(move) // 100
    signed_mod = move % 100 if move >= 0 else -(-move % 100)
    return revolutions + (start != 0 and not 0 < start + signed_mod < 100)


rotations = [int(x.strip('R').replace('L', '-')) for x in aocin(1).split()]

A = sum(x % 100 == 0 for x in accumulate(rotations, initial=50))
assert A == 992

A = sum(hits(r, s % 100) for r, s in zip(rotations, accumulate(rotations, initial=50)))
assert A == 6133

## [Day 2: Gift Shop](https://adventofcode.com/2025/day/2)

An extremely interesting problem for day 2!

Essentially, the problem gives us a list of pairs of numbers $L = ((a_0, b_0), (a_1, b_1), \dots)$ and asks us to find the sum of all numbers in each range that are the result of the concatenation of certain number of "blocks" of (at least two) repeating digits. Those are _invalid ids_.

So, for example, $121212$ would be invalid, because it contains the block $12$ three times. In the first part of the problem, only numbers made of exactly two of those blocks are taken into account, and in the second part the restriction is lifted. We'll analyze the second part as it's a straightforward extension of the first one.

### Solution

Let $N$ be a number of $\delta_N$ digits. $N$ is invalid as per the definition above if and only if for some $k$, with $k | \delta_N$, it can be written as the concatenation of $\delta_N / k$ blocks of $k$ digits each.
Let this block be composed by digits $c_1 c_2 \dots c_k$, where juxtaposition denotes concatenation in base 10 representation.
This would mean that $N$ could be written as:

$$
N = c_1 c_2 \dots c_k \dots c_1 c_2 \dots c_k \dots \dots c_1 c_2 \dots c_k
$$

where the block $c_1 c_2 \dots c_k$ is repeated $\delta_N / k$ times. But this means that:

$$
N = c_1 c_2 \dots c_k \frac{10^{\delta_N} - 1}{10^k - 1}
$$

That is, $N$ is a multiple of a "magic number"

$$
\mu(\delta_N, k) = \frac{10^{\delta_N} - 1}{10^k - 1}
$$

Our second observation is that it's easy to sum the multiples of $n$ from $a$ to $b$, with $a \le b$.
The least of such multiples is $\left \lceil a / n \right \rceil$ and the greatest of them is $\left \lfloor b / n \right \rfloor$, therefore
they form an arithmetic progression of known length and difference:

$$
\sum_{h=a}^{b} \left[ n | h \right] h =
\frac{1}{2} n
\left( \left \lceil a / n \right \rceil + \left \lfloor b / n \right \rfloor \right)
\left( \left \lfloor b / n \right \rfloor - \left \lceil a / n \right \rceil + 1 \right)
$$

Therefore, for a fixed number of digits $\delta_N$ and for whatever interval $\left(a_i, b_i \right)$, if $a_i$ and $b_i$ have the same number of digits, we have an efficient algorithm to find all invalid numbers, as follows:

- Determine the divisors of $\delta_N$, excluding $\delta_N$ itself. Those are our candidate values for $k$.

- For each of them, find the sum of all multiples of $\mu(\delta_N, k)$ ranging from $a_i$ to $b_i$.

It would be tempting to say that we're done, but there's a minor problem. Numbers that are multiples of _both_ $\mu(\delta_N, k_1)$ and $\mu(\delta_N, k_2)$ for
two different values of $k_1, k_2$ have been counted twice. But it's easy enough to solve: we can perform inclusion-exclusion on the set of the divisors of
$\delta_N$.

To conclude, there's a second minor problem: $a_i$ and $b_i$ could have a different number of digits. But this is easily solved: we can "split" our interval in
two subintervals each of which only contains numbers with the same number of digits. For example, we can solve the problem separately for $(80, 99)$ and
$(100, 110)$ and combine the results to obtain the final result for the interval $(80, 110)$.

### Complexity analysis

The algorithm is very efficient and terminates instantly on normal hardware. It's interesting to analyze its theoretical complexity. Because we can cache intermediate results, it becomes difficult to give a straightforward analysis on a complex case, as it would require making assumptions on the distribution of input intervals. We'll simplify the analysis, assuming to run our algorithm on an interval bounded by two integers $(a, b)$ of length $n$ and $m$ respectively.
In other words, $10^{n-1} \le a \lt 10^n$, $10^{m-1} \le b \lt 10^m$.

We'll split the interval in $n-m$ subintervals as per our trick, each representing a different number of digits.
For the subinterval that represents $h$ digits, we'll have to perform a number of operations that's proportional to... $2^{d(h)}$ where $d(h)$ is the number of
divisors of $h$. This is because inclusion-exclusion effectively requires iterating on the powerset:

$$
\sum_{i=1}^{d(h)} \binom{d(h)}{i} = 2^{d(h)}-1
$$

Asymptotic bounds for the number of divisors of $h$ are well-known in literature. In particular, by Dirichlet's divisor problem:

$$
\sum_{i=1}^{x} d(i) = x \log x + x (2 \gamma - 1) + O(\sqrt{x})
$$

The time complexity $T(n)$ of our algorithm is therefore bounded by:

$$
T(n) = \sum_{i=m}^{n} 2^{d(i)} \le \sum_{i=1}^{n} 2^{d(i)}
$$

By Jensen's inequality:

$$
\frac{T(n)}{n} = \frac{1}{n} \sum_{i=1}^{n} 2^{d(i)} \ge 2 ^ {\frac{1}{n} \sum_{i=1}^n d(i)}
$$

Which means:

$$
T(n) \in O(n 2^{\log n}) = O(n^2)
$$

Which doesn't seem great in theory, although the complexity is probably dominated by numbers divisible by a lot of small primes.

In [3]:
@cache
def sum_multiples_of(a, b, n):
    lo = (a - 1) // n + 1
    hi = b // n
    return n * (hi - lo + 1) * (lo + hi) // 2


@cache
def magic_multiple(l, n):
    return (10 ** l - 1) // (10 ** n - 1)


@cache
def invalid_ids(a, b):
    da = len(str(a))
    db = len(str(b))

    low = (
        sum_multiples_of(a, min(10**da-1, b), magic_multiple(da, da//2))
        if da % 2 == 0 else 0
    )

    if da == db:
        return low

    high = (
        sum_multiples_of(max(10**(db-1), a), b, magic_multiple(db, db//2))
        if db % 2 == 0 else 0
    )

    return low + high + sum(
        invalid_ids(max(10**(n-1), a), min(10**n-1, b))
        for n in range(da+1, db)
    )


@cache
def invalid_ids_2(a, b):
    da = len(str(a))
    db = len(str(b))

    da_divisors = {j for j in range(1, da) if da%j == 0}
    low = sum(
        (-1) ** (h + 1) * sum(
            sum_multiples_of(
                a, min(10**da-1, b),
                lcm(*(magic_multiple(da, u) for u in l))
            )
            for l in combinations(da_divisors, h)
        )
        for h in range(1, len(da_divisors)+1)
    )

    if da == db:
        return low

    db_divisors = {j for j in range(1, db) if db%j == 0}
    high = sum(
        (-1) ** (h + 1) * sum(
            sum_multiples_of(
                max(10**(db-1), a), b,
                lcm(*(magic_multiple(db, u) for u in l))
            )
            for l in combinations(db_divisors, h)
        )
        for h in range(1, len(db_divisors)+1)
    )

    return low + high + sum(
        invalid_ids_2(max(10**(n-1), a), min(10**n-1, b))
        for n in range(da+1, db)
    )


ids = [tuple(map(int, x.split('-'))) for x in aocin(2).split(',')]

A = sum(invalid_ids(a, b) for a, b in ids)
assert A == 5398419778

A = sum(invalid_ids_2(a, b) for a, b in ids)
assert A == 15704845910

## [Day 3: Lobby](https://adventofcode.com/2025/day/3)

In [4]:
def max_voltage(battery, banks=2):
    current = []
    for n, u in enumerate(battery):
        while (
            current
            and current[-1] < u
            and len(battery) - n + len(current) > banks
        ):
            current.pop()
        if len(current) < banks:
            current.append(u)
    return int(''.join(current))


batteries = aocin(3).split()

A = sum(max_voltage(b) for b in batteries)
assert A == 17087

A = sum(max_voltage(b, banks=12) for b in batteries)
assert A == 169019504359949

## [Day 4 - Printing Department](https://adventofcode.com/2025/day/4)

Nothing particularly hard today, ~~a priority queue straightforwarldy gives a $O(n \log n)$ solution~~.

EDIT: I'm dumb. A simple queue is sufficient, dropping the time to $O(n)$.

In [5]:
def adjacent(y, x):
    yield from (
        (y + dy, x + dx)
        for dy in (0, 1, -1)
        for dx in (0, 1, -1)
        if dy != 0 or dx != 0
    )


def accessible_rolls(rolls):
    prios, adj = {}, {}
    for r in rolls:
        neighbors = rolls & set(adjacent(*r))
        prios[r] = len(neighbors)
        adj[r] = neighbors

    Q = [r for r, n in prios.items() if n < 4]
    removed = 0
    while Q:
        r = Q.pop()
        removed += 1
        for y, x in adj[r]:
            prios[(y, x)] -= 1
            if prios[(y, x)] == 3:
                Q.append((y, x))

    return removed


rolls = {
    (y, x)
    for y, row in enumerate(aocin(4).split())
    for x, col in enumerate(row)
    if row[x] == '@'
}

A = sum(len(rolls & set(adjacent(*r))) < 4 for r in rolls)
assert A == 1486

A = accessible_rolls(rolls)
assert A == 9024

## [Day 5 - Cafeteria](https://adventofcode.com/2025/day/5)

I thought I had to implement an interval tree, but it didn't turn out to be necessary :)

For part 1, we are given a set of intervals and a set of point queries. The problem asks us to determine how many of those points are part of any interval. In the problem, the intervals are inclusive, so we are switching to the _obviously better_[1] "left inclusive, right exclusive" notation :). The canonical solution is to construct an interval tree, which takes time $O(n \log n)$ where $n$ is the number of intervals, and then perform $k$ membership queries, each of which takes time $O(\log n)$.

But a far simpler implementation with the same computational complexity (and probably better performace in practice since it uses simpler data structures) is this: first, construct a list of "merged" intervals, that is, of pairwise disjoint intervals that cover exactly the same ranges.
To do this, it's sufficient to sort the intervals on their left bound and merge them pairwise left to right. Sorting takes time $O(n \log n)$ and
merging requires $O(n)$ constant checks, for an overall complexity of $O(n \log n)$.
But if we have a _sorted_ list of pairwise disjoint intervals, membership queries can be performed with a simple binary search. This results in the same asymptotic complexity as the traditional method: $O(n \log n)$ preprocessing time and $O(\log n)$ for each query.

The nice thing it that this also implicitly solves part 2: the total size covered by pairwise disjoint intervals is trivial to compute. A rare problem where part 1 is more interesting than part 2!

---

[1] https://www.cs.utexas.edu/~EWD/ewd08xx/EWD831.PDF

In [6]:
def merge_intervals(intervals):
    it = iter(sorted(intervals, key=lambda u: u.start))
    current = next(it, None)
    for iv in it:
        if not current or current.stop < iv.start:
            yield current
            current = iv
        current = range(current.start, max(current.stop, iv.stop))
    if current: yield current

def intersects(intervals, query):
    idx = bisect_right(intervals, query, key=lambda u: u.start)
    if idx == 0: return False
    return query in intervals[idx - 1]


intervals, queries = aocin(5).split('\n\n')
intervals = [
    range(a, b+1)
    for a, b in (map(int, u.split('-')) for u in intervals.split('\n'))
]
queries = list(map(int, queries.split('\n')))
merged_intervals = list(merge_intervals(intervals))

A = sum(intersects(merged_intervals, q) for q in queries)
assert A == 896

A = sum(len(iv) for iv in merged_intervals)
assert A == 346240317247002

## [Day 6 - Trash Compactor](https://adventofcode.com/2025/day/6)

In [7]:
def transform_rtl(args):
    current = []
    for t in zip_longest(*args, fillvalue=' '):
        try:
            current.append(int(''.join(t)))
        except ValueError:
            yield current
            current = []
    if current: yield current


*args, ops = aocin(6).split('\n')
args_r = [list(map(int, u.split())) for u in args]
args_t = list(transform_rtl(args))
ops = [add if u == '+' else mul for u in ops.split()]

A = sum(reduce(op, a) for a, op in zip(list(zip(*args_r)), ops))
assert A == 5060053676136

A = sum(reduce(op, a) for a, op in zip(args_t, ops))
assert A == 9695042567249

## [Day 7 - Laboratories](https://adventofcode.com/2025/day/7)

In [8]:
def propagate(splitters):
    first, *rest = splitters
    n_splits = 0

    for s in rest:
        i = first & s
        n_splits += len(i)
        new = set()
        for u in i:
            new.add(u+1), new.add(u-1)
        first = first - s | new

    return n_splits


def split_time(splitters):
    def perform_step(c, s):
        n = Counter()
        for u in c:
            if u in s:
                n[u+1] += c[u]
                n[u-1] += c[u]
            else:
                n[u] += c[u]
        return n

    first, *rest = splitters
    return reduce(perform_step, rest, Counter(first)).total()


lines = aocin(7).split()
splitters = [{n for n, k in enumerate(l) if k in 'S^'} for l in lines]
splitters = list(filter(None, splitters))

A = propagate(splitters)
assert A == 1633

A = split_time(splitters)
assert A == 34339203133559

## [Day 8 - Playground](https://adventofcode.com/2025/day/8)

A textbook problem: construct the minimum spanning tree using Kruskal's algorithm. We get to practice union-find data structures.

In [9]:
class UnionFind:
    @dataclass(slots=True)
    class Member:
        item: int
        parent: int
        size: int

    def __init__(self, size):
        self.d = [UnionFind.Member(u, u, 1) for u in range(size)]

    def find(self, u):
        m = self.d[u]
        if m.parent == u: return u
        m.parent = self.find(m.parent)
        return m.parent

    def union(self, u, v):
        if (m := self.find(u)) == (n := self.find(v)): return False
        m, n = self.d[m], self.d[n]
        if m.size < n.size: m, n = n, m
        m.parent = n.item
        n.size += m.size
        return True


def make_connections(points, distances, largest):
    uf = UnionFind(len(points))
    for u, v, _ in distances: uf.union(u, v)
    return prod(
        sorted(
            (u.size for u in uf.d if u.item == u.parent),
            reverse=True
        )[:largest]
    )


def last_junctions(points, distances):
    uf = UnionFind(len(points))
    count = len(points) - 1
    for u, v, _ in distances:
        count -= uf.union(u, v)
        if not count: break
    return points[u][0] * points[v][0]


points = list(tuple(map(int, u.split(','))) for u in aocin(8).split())
distances = sorted((
    (a, b,    (points[a][0]-points[b][0])**2
            + (points[a][1]-points[b][1])**2
            + (points[a][2]-points[b][2])**2)
    for a, b in combinations(range(len(points)), 2)),
    key=lambda u: u[2]
)

A = make_connections(points, distances[:1000], largest=3)
assert A == 133574

A = last_junctions(points, distances)
assert A == 2435100380

For part 2, there's a fun optimization: the only edges that we are ever going to use are part of the minimum spanning tree, but the minimum spanning tree is a subgraph of the Delaunay triangulation. We can only construct distances between those pairs of edges.

In [10]:
points = list(tuple(map(int, u.split(','))) for u in aocin(8).split())
t_edges = sorted((
    (u, v, (points[u][0]-points[v][0])**2
         + (points[u][1]-points[v][1])**2
         + (points[u][2]-points[v][2])**2)
    for l in Delaunay(points).simplices
    for u, v in combinations(l, 2)
), key=lambda u: u[2])

A = last_junctions(points, t_edges)
assert A == 2435100380

## [Day 9 - Movie Theater](https://adventofcode.com/2025/day/9)

In [11]:
def is_valid(x0, y0, x1, y1, horz, vert):
    if x0 > x1: x0, x1 = x1, x0
    if y0 > y1: y0, y1 = y1, y0
    l = bisect_left(horz, y0, key=lambda u: u[0])
    r = bisect_right(horz, y1, key=lambda u: u[0])
    for y, s, e in horz[l:r]:
        if y0 < y < y1 and min(s, e) < x1 and max(s, e) > x0:
            return False
    l = bisect_left(vert, x0, key=lambda u: u[0])
    r = bisect_right(vert, x1, key=lambda u: u[0])
    for x, s, e in vert[l:r]:
        if x0 < x < x1 and min(s, e) < y1 and max(s, e) > y0:
            return False
    return True


points = [tuple(map(int, u.split(','))) for u in aocin(9).split()]
bounds = list(pairwise(points)) + [(points[-1], points[0])]
horz = sorted((u[0][1], u[0][0], u[1][0]) for u in bounds if u[0][1] == u[1][1])
vert = sorted((u[0][0], u[0][1], u[1][1]) for u in bounds if u[0][0] == u[1][0])

A = max(
    (abs(x0-x1)+1) * (abs(y0-y1)+1)
    for (x0, y0), (x1, y1) in combinations(points, 2)
)
assert A == 4740155680

A = max(
    (abs(x0-x1)+1) * (abs(y0-y1)+1)
    for (x0, y0), (x1, y1) in combinations(points, 2)
    if is_valid(x0, y0, x1, y1, horz, vert)
)
assert A == 1543501936

## [Day 10 - Factory](https://adventofcode.com/2025/day/10)

I wonder what was the intended solution today? Those are a set of integer linear programs, were we supposed to write a branch-and-bound implementation from scratch?

In [12]:
def parse_instance(s):
    goal, *buttons, energy = s.split()
    goal = set(u for u, v in enumerate(goal.strip('[]')) if v == '#')
    buttons = list(map(
        lambda u: set(map(int, u.strip('()').split(','))), buttons
    ))
    energy = tuple(map(int, energy.strip('{}').split(',')))
    return goal, buttons, energy


def solve_instance(goal, buttons, _):
    for i in range(len(buttons)):
        for s in combinations(buttons, i):
            if reduce(xor, s, set()) == goal:
                return i


def solve_instance_energy(_, buttons, energy):
    button_eq = list(zip(*(
        tuple(u in b for u in range(len(energy)))
        for b in buttons
    )))
    res = milp([1] * len(buttons), integrality=1, constraints=(button_eq, energy, energy))
    return res.fun


instances = aocin(10).split('\n')

A = sum(solve_instance(*u) for u in map(parse_instance, instances))
assert A == 491

A = sum(solve_instance_energy(*u) for u in map(parse_instance, instances))
assert A == 20617

## [Day 11 - Reactor](https://adventofcode.com/2025/day/11)

Easy problem but interesting idea. For the problem to have a finite solution, the graph must be acyclic, which makes both parts trivial. (This is not stated as an hypothesis in the problem statement, though. I assume it's an omission?). Technically, we could even slightly relax the acyclic condition: it's sufficient for the graph not to have any cycles on any path from the starting node to the ending node. This would make the implementation slightly more complicated, but it would still be a linear time algorithm: cycles can be detected easily enough keeping some extra information while performing the DFS.

Another interesting generalization of part 2 is this: consider an acyclic graph $G = (V, E)$, a starting node $S \in V$, a final node $F \in V$ and a list of "intermediate" nodes $I_1, I_2, \dots I_n \in V$. There is a linear time algorithm to compute how many paths go from $S$ to $F$, passing from each of the $I_h$ in any order. This is because if $G$ is acyclic, then there is only one order in which the intermediate nodes can be reached, the same order they appear in a topological sort of $G$.

In [13]:
def paths(G, src, dest):
    @cache
    def _paths_rec(src, dest):
        if src == dest: return 1
        return sum(_paths_rec(u, dest) for u in G.get(src, []))
    return _paths_rec(src, dest)

G = {
    u[0].strip(':'): u[1:]
    for u in map(str.split, aocin(11).split('\n'))
}


A = paths(G, 'you', 'out')
assert A == 662

A = sum(
    paths(G, 'svr', a) * paths(G, a, b) * paths(G, b, 'out')
    for a, b in (('fft', 'dac'), ('dac', 'fft'))
)
assert A == 429399933071120

## [Day 12 - Christmas Tree Farm](https://adventofcode.com/2025/day/12)b

As usual for Advent of Code, the problem statement is just trolling. All test cases are trivially solved by checking the bounds. We would need a constraint satisfaction solver otherwise.

In [14]:
def parse_region(r):
    size, count = r.split(':')
    return (tuple(map(int, size.split('x'))),
            tuple(map(int, count.split())))

def parse_present(p):
    return p, p.count('#')

def bound(region, presents):
    (x, y), c = region
    if x*y < sumprod(c, (p[1] for p in presents)):
        return False
    elif x*y >= 9 * sum(c):
        return True


*presents, regions = aocin(12).split('\n\n')
regions = list(map(parse_region, regions.split('\n')))
presents = list(map(parse_present, presents))

A = sum(bound(r, presents) for r in regions)
assert A == 589